In [1]:
import sqlite3
import pandas as pd
import numpy as np
import sys
import glob, os
from pyteomics import mgf

In [2]:
RT_LIMIT_LOWER = 3000
RT_LIMIT_UPPER = 3060

In [3]:
BASE_MAXQUANT_DIR = '/Users/darylwilding-mcbride/Downloads/maxquant_results'
MAXQUANT_TXT_DIR = '{}/txt'.format(BASE_MAXQUANT_DIR)
ALLPEPTIDES_FILENAME = '{}/allPeptides.txt'.format(MAXQUANT_TXT_DIR)
APL_DIR = '{}/andromeda'.format(BASE_MAXQUANT_DIR)

In [4]:
MIN_ISOTOPE_CORRELATION = 0.9

allpeptides_df = pd.read_csv(ALLPEPTIDES_FILENAME, sep='\t')
allpeptides_df.rename(columns={'Number of isotopic peaks':'isotope_count', 'm/z':'mz', 'Number of data points':'number_data_points', 'Intensity':'intensity', 'Ion mobility index':'scan', 'Ion mobility index length':'scan_length', 'Ion mobility index length (FWHM)':'scan_length_fwhm', 'Retention time':'rt', 'Retention length':'rt_length', 'Retention length (FWHM)':'rt_length_fwhm', 'Charge':'charge_state', 'Number of pasef MS/MS':'number_pasef_ms2_ids', 'Pasef MS/MS IDs':'pasef_msms_ids', 'MS/MS scan number':'msms_scan_number', 'Isotope correlation':'isotope_correlation'}, inplace=True)
allpeptides_df = allpeptides_df[allpeptides_df.intensity.notnull() & (allpeptides_df.number_pasef_ms2_ids > 0) & (allpeptides_df.msms_scan_number >= 0) & allpeptides_df.pasef_msms_ids.notnull() & (allpeptides_df.isotope_correlation >= MIN_ISOTOPE_CORRELATION) & (allpeptides_df.rt >= RT_LIMIT_LOWER) & (allpeptides_df.rt <= RT_LIMIT_UPPER)].copy()


In [5]:
allpeptides_df.msms_scan_number = allpeptides_df.msms_scan_number.apply(lambda x: int(x))

In [6]:
allpeptides_df.msms_scan_number.max()

100522

In [7]:
allpeptides_df.tail()

,Raw file,charge_state,mz,Mass,Resolution,number_data_points,Number of frames,isotope_count,isotope_correlation,Mass fractional part,...,Min frame index,Max frame index,scan,scan_length,scan_length_fwhm,intensity,Intensities,number_pasef_ms2_ids,pasef_msms_ids,msms_scan_number
325494,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,688.35495,2062.0430,24291.598528,10295,18,5,0.997244,0.043033,...,2563,2580,717,72,33,165630.0,NaN,2,200445;200455,83450
325495,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,711.68902,2132.0452,24310.732974,22997,28,5,0.997653,0.045229,...,2527,2554,717,75,30,207660.0,NaN,4,197557;197565;197574;198693,85780
325604,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,713.28610,2136.8365,24483.385523,18700,30,5,0.994567,0.836482,...,2536,2565,714,72,27,128140.0,NaN,2,198173;198184,85899
326521,HeLa_20KInt_2KIT_Slot1-46_01_1179,3,639.94334,1916.8082,24514.692836,27504,27,5,0.998548,0.808196,...,2571,2597,723,174,33,431340.0,NaN,4,201146;201155;201345;201353,77534
327447,HeLa_20KInt_2KIT_Slot1-46_01_1179,2,673.38573,1344.7569,23387.664514,1698,12,2,0.954187,0.756915,...,2551,2562,771,54,45,23594.0,NaN,1,199498,40002


In [8]:
allpeptides_df.columns

Index(['Raw file', 'charge_state', 'mz', 'Mass', 'Resolution',
       'number_data_points', 'Number of frames', 'isotope_count',
       'isotope_correlation', 'Mass fractional part', 'Mass deficit', 'rt',
       'rt_length', 'rt_length_fwhm', 'Min frame index', 'Max frame index',
       'scan', 'scan_length', 'scan_length_fwhm', 'intensity', 'Intensities',
       'number_pasef_ms2_ids', 'pasef_msms_ids', 'msms_scan_number'],
      dtype='object')

In [16]:
def collate_spectra_for_feature(ms1_d, ms2_df):
    # append the monoisotopic and the ms2 fragments to the list for MGF creation
    pairs_df = ms2_df[['mz', 'intensity']].copy().sort_values(by=['intensity'], ascending=False)
    spectrum = {}
    spectrum["m/z array"] = pairs_df.mz.values
    spectrum["intensity array"] = pairs_df.intensity.values
    params = {}
    params["TITLE"] = "RawFile: {} Index: {} Charge: {} FeatureIntensity: {} RtApex: {}".format(ms1_d['raw_file'], ms1_d['mq_index'], ms1_d['charge'], ms1_d['intensity'], round(ms1_d['rt_apex'],2))
    params["INSTRUMENT"] = "ESI-QUAD-TOF"
    params["PEPMASS"] = "{} {}".format(round(ms1_d['monoisotopic_mz'],6), ms1_d['intensity'])
    params["CHARGE"] = "{}+".format(ms1_d['charge'])
    params["RTINSECONDS"] = "{}".format(round(ms1_d['rt_apex'],2))
    params["SCANS"] = "{}".format(int(ms1_d['rt_apex']))
    spectrum["params"] = params
    return spectrum

In [17]:
# build a list of indexes from the APL files
ms2_peaks = []
apl_indexes = []
for file in glob.glob("{}/*.apl".format(APL_DIR)):
    with open(file, 'r') as f:
        for line in f:
            line = line.rstrip()
            if len(line) > 0:
                if line.startswith("header="):
                    mq_index = int(line.split(' ')[3])
                if line[0].isdigit():
                    line_a = line.split('\t')
                    mz = float(line_a[0])
                    intensity = round(float(line_a[1]))
                    ms2_peaks.append((mz, intensity))
                if line.startswith("peaklist end"):
                    apl_indexes.append((mq_index, ms2_peaks.copy()))
                    del ms2_peaks[:]
                    mq_index = 0
apl_indexes_df = pd.DataFrame(apl_indexes, columns=['mq_index','ms2_peaks'])

In [18]:
apl_indexes_df.head()

,mq_index,ms2_peaks
0,18180,"[(198.12835, 15), (226.11905, 16), (235.11728,..."
1,18181,"[(204.08961, 10), (231.11226, 21), (235.04234,..."
2,18182,"[(166.28784, 9), (171.15202, 26), (195.07293, ..."
3,18183,"[(212.10498, 15), (217.11978, 9), (237.10405, ..."
4,18184,"[(226.11908, 16), (227.12076, 10), (244.16614,..."


In [19]:
mgf_spectra = []
for idx,row in allpeptides_df.iterrows():
    mq_index = row.msms_scan_number
    ms1_d = {'monoisotopic_mass':row.Mass, 
             'charge':row.charge_state, 
             'monoisotopic_mz':row.mz, 
             'intensity':int(row.intensity), 
             'scan_apex':row.scan, 
             'rt_apex':row.rt,
             'raw_file':row['Raw file'],
             'mq_index':mq_index}
    df = apl_indexes_df[apl_indexes_df.mq_index == mq_index]
    if len(df) == 1:
        ms2_peaks_df = pd.DataFrame(df.ms2_peaks.values[0], columns=['mz','intensity'])
        feature_spectra = collate_spectra_for_feature(ms1_d, ms2_peaks_df)
        mgf_spectra.append(feature_spectra)
    else:
        print("no match for mq_index {}".format(mq_index))

In [20]:
len(mgf_spectra)

858

In [21]:
allpeptides_df.msms_scan_number.nunique()

858

In [22]:
MGF_FILE_NAME = '/Users/darylwilding-mcbride/Downloads/mq_features.mgf'
# generate the MGF for all the features
print("generating the MGF: {}".format(MGF_FILE_NAME))
if os.path.isfile(MGF_FILE_NAME):
    os.remove(MGF_FILE_NAME)
f = mgf.write(output=MGF_FILE_NAME, spectra=mgf_spectra)


generating the MGF: /Users/darylwilding-mcbride/Downloads/mq_features.mgf
